In [1]:
import warnings; warnings.filterwarnings('ignore')

In [2]:
import ibis
from rbc.heavydb import RemoteHeavyDB

In [3]:
# NBVAL_IGNORE_OUTPUT
heavydb = RemoteHeavyDB()  # RemoteHeavyDB(user=, password=, host=, port=, dbname=)
heavydb_ibis = ibis.omniscidb.connect(user=heavydb.user, password=heavydb.password,
                                      host=heavydb.host, port=heavydb.port, database=heavydb.dbname)

ERROR 1: PROJ: proj_create_from_database: Open of /home/guilhermeleobas/miniconda3/envs/rbc/share/proj failed


In [4]:
schema = ibis.schema([('x', 'float'), ('y', 'double'), ('i', 'int32'), ('j', 'int64')])
print(schema)

ibis.Schema {  
  x  float32
  y  float64
  i  int32
  j  int64
}


In [5]:
# Create test table
heavydb_ibis.drop_table('mytable')
heavydb_ibis.create_table('mytable', schema=schema)
mytable = heavydb_ibis.table('mytable')
for i in range(5):
    heavydb.sql_execute('insert into mytable values (%s, %s, %s, %s)' % (i, i+10, i+100, i+1000))
mytable.execute()

,x,y,i,j
0,0.0,10.0,100,1000
1,1.0,11.0,101,1001
2,2.0,12.0,102,1002
3,3.0,13.0,103,1003
4,4.0,14.0,104,1004


In [6]:
# Define and register OmnisciDB Runtime UDF
@heavydb('int32(int32)')
def incr(x):
    return x + 1
@heavydb('int64(int64)')
def incr(x):
    return x + 10
heavydb.register()

In [7]:
t = heavydb_ibis.sql('select i, incr(i), j, incr(j) from mytable')
expr = t[t.i < 103]
print(expr.compile())

SELECT *
FROM (
  select i, incr(i), j, incr(j) from mytable
) t0
WHERE "i" < 103


In [8]:
expr.execute()

,i,EXPR$1,j,EXPR$3
0,100,101,1000,1010
1,101,102,1001,1011
2,102,103,1002,1012
